Introduction and objectives

# Chapter 2 – Recommendation Systems (Practice Notebook)

This notebook is based on *A Programmer's Guide to Data Mining* – Chapter 2 (Recommendation Systems).

**Objectives:**

1. Load the chapter resources with explicit encoding (UTF-8).
2. Define a small rating dataset for users and items.
3. Implement distance/similarity functions:
   - Manhattan distance
   - Euclidean distance
   - Minkowski distance
   - Pearson correlation
4. Implement a simple k-nearest-neighbours (k-NN) recommender:
   - Find users most similar to a target user.
   - Predict ratings for items the target user has not rated.
5. Run a few experiments and interpret the results.


Loading the chapter resources (with explicit encoding)

In [5]:
from pathlib import Path

BASE_DIR = Path(r"D:\daniel\JupyterProjects\pg2dm\ch2")
DATA_DIR = BASE_DIR / "data"

readme_path = DATA_DIR / "readmech2_utf8.txt"

if readme_path.exists():
    with open(readme_path, encoding="utf-8") as f:
        readme_text = f.read()
    print("Loaded readmech2_utf8.txt (first 400 chars):")
    print("-" * 80)
    print(readme_text[:400])
else:
    print(f"WARNING: {readme_path} not found. Skipping readme preview.")


Loaded readmech2_utf8.txt (first 400 chars):
--------------------------------------------------------------------------------
Contents

    How a recommendation system works.
    How social filtering works
    How to find similar items
    Manhattan distance
    Euclidean distance
    Minkowski distance
    Pearson Correlation Coefficient
    Cosine similarity
    Implementing k-nearest neighbors in Python
    The Book Crossing dataset
The PDF of the Chapter
Python code

The code for the initial Python example: filtering


In [13]:
from pathlib import Path
import sys

BASE_DIR = Path(r"D:\daniel\JupyterProjects\pg2dm\ch2")
SRC_DIR = BASE_DIR / "src"

# Allow importing ch2_recommender package
if str(SRC_DIR) not in sys.path:
    sys.path.append(str(SRC_DIR))

from ch2_recommender.distances import (
    RatingDict,
    manhattan_distance,
    euclidean_distance,
    minkowski_distance,
    pearson_correlation,
    manhattan_similarity,
    pearson_similarity,
)
from ch2_recommender.recommender import (
    neighbours,
    recommend_user_based,
    distance_matrix,
)


Rating dataset (small hardcoded example)

In [6]:
# Simple user–item rating dictionary.
# Structure:
#   ratings[user][item] = rating (1–5)

ratings = {
    "Alice": {
        "Band A": 5,
        "Band B": 3,
        "Band C": 4,
    },
    "Bob": {
        "Band A": 3,
        "Band B": 4,
        "Band D": 2,
    },
    "Carol": {
        "Band B": 2,
        "Band C": 5,
        "Band D": 5,
    },
    "Dave": {
        "Band A": 4,
        "Band C": 3,
        "Band D": 4,
    },
}

users = list(ratings.keys())
items = sorted({item for r in ratings.values() for item in r.keys()})

print("Users:", users)
print("Items:", items)


Users: ['Alice', 'Bob', 'Carol', 'Dave']
Items: ['Band A', 'Band B', 'Band C', 'Band D']


Distance / similarity functions

In [7]:
import math
from typing import Dict, Iterable

RatingDict = Dict[str, Dict[str, float]]


def shared_items(ratings: RatingDict, u1: str, u2: str) -> Iterable[str]:
    """Return the set of items rated by both u1 and u2."""
    return set(ratings[u1].keys()).intersection(ratings[u2].keys())


def manhattan_distance(ratings: RatingDict, u1: str, u2: str) -> float:
    """Manhattan (L1) distance between two users over shared items."""
    common = shared_items(ratings, u1, u2)
    if not common:
        return math.inf
    return sum(abs(ratings[u1][i] - ratings[u2][i]) for i in common)


def euclidean_distance(ratings: RatingDict, u1: str, u2: str) -> float:
    """Euclidean (L2) distance."""
    common = shared_items(ratings, u1, u2)
    if not common:
        return math.inf
    return math.sqrt(sum((ratings[u1][i] - ratings[u2][i]) ** 2 for i in common))


def minkowski_distance(ratings: RatingDict, u1: str, u2: str, p: float = 2.0) -> float:
    """General Minkowski distance (p >= 1)."""
    if p <= 0:
        raise ValueError("p must be > 0")
    common = shared_items(ratings, u1, u2)
    if not common:
        return math.inf
    return sum(abs(ratings[u1][i] - ratings[u2][i]) ** p for i in common) ** (1.0 / p)


def pearson_correlation(ratings: RatingDict, u1: str, u2: str) -> float:
    """Pearson correlation coefficient between two users."""
    common = list(shared_items(ratings, u1, u2))
    n = len(common)
    if n == 0:
        return 0.0

    sum1 = sum(ratings[u1][i] for i in common)
    sum2 = sum(ratings[u2][i] for i in common)

    sum1_sq = sum(ratings[u1][i] ** 2 for i in common)
    sum2_sq = sum(ratings[u2][i] ** 2 for i in common)

    p_sum = sum(ratings[u1][i] * ratings[u2][i] for i in common)

    num = p_sum - (sum1 * sum2 / n)
    den = math.sqrt((sum1_sq - sum1 ** 2 / n) * (sum2_sq - sum2 ** 2 / n))

    if den == 0:
        return 0.0
    return num / den


Test them:

In [8]:
for u1 in users:
    for u2 in users:
        if u1 >= u2:
            continue
        print(f"{u1} – {u2}: Manhattan={manhattan_distance(ratings, u1, u2):.2f}, "
              f"Pearson={pearson_correlation(ratings, u1, u2):.2f}")


Alice – Bob: Manhattan=3.00, Pearson=-1.00
Alice – Carol: Manhattan=2.00, Pearson=1.00
Alice – Dave: Manhattan=2.00, Pearson=1.00
Bob – Carol: Manhattan=5.00, Pearson=-1.00
Bob – Dave: Manhattan=3.00, Pearson=0.00
Carol – Dave: Manhattan=3.00, Pearson=0.00


This cell mirrors the math from the book and is easy to migrate later into src/ch2_recommender/distances.py.

Finding nearest neighbours

In [9]:
from typing import Callable, List, Tuple

SimilarityFunc = Callable[[RatingDict, str, str], float]


def manhattan_similarity(ratings: RatingDict, u1: str, u2: str) -> float:
    """Convert Manhattan distance into a similarity (smaller distance => larger similarity)."""
    d = manhattan_distance(ratings, u1, u2)
    if d == math.inf:
        return 0.0
    return 1.0 / (1.0 + d)


def neighbours(
    ratings: RatingDict,
    target_user: str,
    similarity_fn: SimilarityFunc,
    top_n: int = 3,
) -> List[Tuple[float, str]]:
    """Return top_n neighbours (similarity, user) for target_user."""
    scores = []
    for other in ratings:
        if other == target_user:
            continue
        sim = similarity_fn(ratings, target_user, other)
        scores.append((sim, other))
    scores.sort(reverse=True, key=lambda x: x[0])
    return scores[:top_n]


# Example: neighbours using inverse Manhattan similarity
target = "Alice"
print(f"Nearest neighbours of {target} (Manhattan-based):")
for sim, u in neighbours(ratings, target, manhattan_similarity, top_n=3):
    print(f"  {u}: similarity={sim:.3f}")


Nearest neighbours of Alice (Manhattan-based):
  Carol: similarity=0.333
  Dave: similarity=0.333
  Bob: similarity=0.250


You can later plug in a Pearson-based similarity: lambda r, u1, u2: pearson_correlation(r, u1, u2).

Simple recommendation function

We implement a basic user-based collaborative filtering:

For each item not rated by target user:

Look at neighbours who rated that item.

Compute a weighted average of their ratings, weighting by similarity.

In [10]:
def recommend_user_based(
    ratings: RatingDict,
    target_user: str,
    similarity_fn: SimilarityFunc,
    k: int = 3,
    min_sim: float = 0.0,
) -> List[Tuple[float, str]]:
    """
    Recommend items to target_user based on k nearest neighbours.

    Returns list of (predicted_rating, item), sorted by rating desc.
    """
    # 1. Get k neighbours
    neighs = neighbours(ratings, target_user, similarity_fn, top_n=k)

    # 2. Items not yet rated by target user
    target_items = set(ratings[target_user].keys())
    all_items = {i for r in ratings.values() for i in r.keys()}
    candidate_items = all_items - target_items

    scores = {}  # item -> weighted sum
    sim_sums = {}  # item -> sum of similarities

    for sim, other in neighs:
        if sim <= min_sim:
            continue
        for item, r in ratings[other].items():
            if item in target_items:
                continue
            scores.setdefault(item, 0.0)
            sim_sums.setdefault(item, 0.0)
            scores[item] += sim * r
            sim_sums[item] += sim

    # 3. Compute predicted ratings
    predictions = []
    for item in scores:
        if sim_sums[item] == 0:
            continue
        predictions.append((scores[item] / sim_sums[item], item))

    predictions.sort(reverse=True, key=lambda x: x[0])
    return predictions


# Example: recommend for Alice using Pearson correlation
def pearson_similarity(ratings: RatingDict, u1: str, u2: str) -> float:
    # Pearson already behaves like a similarity in [-1, 1], but we only want >= 0
    corr = pearson_correlation(ratings, u1, u2)
    return max(0.0, corr)


print("Recommendations for Alice (user-based, Pearson):")
for pred, item in recommend_user_based(ratings, "Alice", pearson_similarity, k=3):
    print(f"  {item}: predicted rating {pred:.2f}")


Recommendations for Alice (user-based, Pearson):
  Band D: predicted rating 4.50


You can now play:

Change target_user.

Change similarity_fn (Manhattan vs Pearson).

Change k and min_sim.

Experiments (Markdown + small cells)

## Experiments

Ideas:

1. Reproduce exactly one of the tables from the book (e.g. distance matrix between all users).
2. Change one user's ratings and see how the neighbours and recommendations change.
3. Add a new user with a few ratings and see what items are recommended.
4. Later: load a larger rating dataset from CSV using `encoding="utf-8"` and apply the same functions.


Example experiment code cell:

In [11]:
def distance_matrix(ratings: RatingDict, distance_fn) -> None:
    users = list(ratings.keys())
    print("Distance matrix:")
    print("       " + "  ".join(f"{u:>7}" for u in users))
    for u1 in users:
        row = [f"{u1:>7}"]
        for u2 in users:
            if u1 == u2:
                row.append("   0.00")
            else:
                d = distance_fn(ratings, u1, u2)
                if d == math.inf:
                    row.append("    inf")
                else:
                    row.append(f"{d:7.2f}")
        print("  ".join(row))


distance_matrix(ratings, manhattan_distance)


Distance matrix:
         Alice      Bob    Carol     Dave
  Alice     0.00     3.00     2.00     2.00
    Bob     3.00     0.00     5.00     3.00
  Carol     2.00     5.00     0.00     3.00
   Dave     2.00     3.00     3.00     0.00


How to evolve this into a proper module

Move the distance/recommendation functions into:

ch2/src/ch2_recommender/distances.py

ch2/src/ch2_recommender/recommender.py

In [12]:
import sys
from pathlib import Path

BASE_DIR = Path(r"D:\daniel\JupyterProjects\pg2dm\ch2")
SRC_DIR = BASE_DIR / "src"
sys.path.append(str(SRC_DIR))

from ch2_recommender.distances import (
    manhattan_distance,
    euclidean_distance,
    minkowski_distance,
    pearson_correlation,
)
from ch2_recommender.recommender import (
    neighbours,
    recommend_user_based,
)


==========

In [14]:
ratings: RatingDict = {
    "Alice": {"Band A": 5, "Band B": 3, "Band C": 4},
    "Bob":   {"Band A": 3, "Band B": 4, "Band D": 2},
    "Carol": {"Band B": 2, "Band C": 5, "Band D": 5},
    "Dave":  {"Band A": 4, "Band C": 3, "Band D": 4},
}


In [15]:
# Distance matrix with Manhattan distance
distance_matrix(ratings, manhattan_distance)


Distance matrix:
         Alice      Bob    Carol     Dave
  Alice     0.00     3.00     2.00     2.00
    Bob     3.00     0.00     5.00     3.00
  Carol     2.00     5.00     0.00     3.00
   Dave     2.00     3.00     3.00     0.00


In [16]:
# Neighbours of Alice using Manhattan-based similarity
for sim, u in neighbours(ratings, "Alice", manhattan_similarity, top_n=3):
    print(u, sim)


Carol 0.3333333333333333
Dave 0.3333333333333333
Bob 0.25


In [17]:
# Recommendations for Alice using Pearson similarity
recs = recommend_user_based(ratings, "Alice", pearson_similarity, k=3)
for score, item in recs:
    print(f"{item}: predicted rating {score:.2f}")


Band D: predicted rating 4.50


Minimal next steps (to keep you out of overwhelm)

For now, ignore all other projects and just do this small checklist:

Create the src/ch2_recommender/ folder and three files above.

Ensure the notebook can import them and run the simple tests.

Once it works, adapt the ratings dictionary to the exact example from Zacharski’s Chapter 2.

Only after that, consider any extension (CSV data, more users, etc.).

This way Chapter 2 becomes one contained “unit”: clear entry, clear exit, and a reusable code asset you can build on for the rest of the book.